In [2]:
import pandas as pd
import numpy as np
import nltk
from collections import Counter
from nltk.corpus import stopwords
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.optimize import minimize
stops = set(stopwords.words("english"))
import xgboost as xgb
from sklearn.cross_validation import train_test_split
import multiprocessing
import difflib
import time

def diff_ratios(st1, st2):
    seq = difflib.SequenceMatcher()
    seq.set_seqs(str(st1).lower(), str(st2).lower())
    return seq.ratio()

def word_match_share(row):
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        return 0
    shared_words_in_q1 = [w for w in q1words.keys() if w in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w in q1words]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2))/(len(q1words) + len(q2words))
    return R

def get_features2(df_features):
    print('nouns...')
    df_features['question1_nouns'] = df_features.question1.map(lambda x: [w for w, t in nltk.pos_tag(nltk.word_tokenize(str(x).lower())) if t[:1] in ['N']])
    df_features['question2_nouns'] = df_features.question2.map(lambda x: [w for w, t in nltk.pos_tag(nltk.word_tokenize(str(x).lower())) if t[:1] in ['N']])
    df_features['z_noun_match'] = df_features.apply(lambda r: sum([1 for w in r.question1_nouns if w in r.question2_nouns]), axis=1)  #takes long
    print('lengths...')
    df_features['z_len1'] = df_features.question1.map(lambda x: len(str(x)))
    df_features['z_len2'] = df_features.question2.map(lambda x: len(str(x)))
    df_features['z_word_len1'] = df_features.question1.map(lambda x: len(str(x).split()))
    df_features['z_word_len2'] = df_features.question2.map(lambda x: len(str(x).split()))
    print('difflib...')
    df_features['z_match_ratio'] = df_features.apply(lambda r: diff_ratios(r.question1, r.question2), axis=1)  #takes long
    print('word match...')
    df_features['z_word_match'] = df_features.apply(word_match_share, axis=1, raw=True)
    print('tfidf...')
    z1 = tfidf.transform(df_features.question1)
    z2 = tfidf.transform(df_features.question2)
    df_features['z_tfidf_sum1'] = z1.sum(axis = 1)
    df_features['z_tfidf_sum2'] = z2.sum(axis = 1)
    df_features['z_tfidf_len1'] = z1.sign().sum(axis = 1)
    df_features['z_tfidf_len2'] = z2.sign().sum(axis = 1)
    df_features['z_tfidf_mean1'] = df_features['z_tfidf_sum1']/df_features['z_tfidf_len1']
    df_features['z_tfidf_mean2'] = df_features['z_tfidf_sum2']/df_features['z_tfidf_len2']
    return df_features.fillna(0.0)

def get_full(train_df, test_df):
    t1 = time.time()
    full = pd.concat((train_df, test_df), axis = 0)
    print(full.shape)
    final = get_features2(full)
    df_train = final.iloc[:train.shape[0], :]
    df_test = final.iloc[train.shape[0]:, :]
    print(df_train.shape, df_test.shape)
    df_train.to_csv('train_the_1owl_features.csv', index=False)
    df_test.to_csv('test_the_1owl_features.csv', index=False)
    print('Time it took:', time.time() - t1)
    return

/home/w/anaconda3/envs/idp3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
src_train = 'df_train_spacy_lemmat.csv'
src_test = 'df_test_spacy_lemmat.csv'

train = pd.read_csv(src_train)
test = pd.read_csv(src_test)

train.fillna('empty', inplace =  True)
test.fillna('empty', inplace = True)

tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1, 1))
#cvect = CountVectorizer(stop_words='english', ngram_range=(1, 1))

tfidf_txt = pd.Series(train['question1'].tolist() + train['question2'].tolist() + test['question1'].tolist() + test['question2'].tolist()).astype(str)
tfidf.fit_transform(tfidf_txt)
#cvect.fit_transform(tfidf_txt)

<5500172x100744 sparse matrix of type '<class 'numpy.float64'>'
	with 28635790 stored elements in Compressed Sparse Row format>

In [5]:
get_full(train, test)

(2750086, 7)
nouns...
lengths...
difflib...
word match...
tfidf...
(404290, 22) (2345796, 22)
Time it took: 5487.7613966465
